# OWSLib versus Birdy

This notebook shows a side-by-side comparison of `owslib.wps.WebProcessingService` and `birdy.WPSClient`. 


In [80]:
from owslib.wps import WebProcessingService
from birdy import WPSClient

url = "http://localhost:5000/wps/WebProcessingService?service=WPS&version=2.0.0&request=GetCapabilities"

wps = WebProcessingService(url)
cli = WPSClient(url=url)

## Displaying available processes

With `owslib`, `wps.processes` is the list of processes offered by the server. 
With `birdy`, the client is like a module with functions. So you just write `cli.` and press `Tab` to display a drop-down menu of processes. 

In [81]:
wps.processes

[<owslib.wps.Process ultimate_question>,
 <owslib.wps.Process sleep>,
 <owslib.wps.Process nap>,
 <owslib.wps.Process bbox>,
 <owslib.wps.Process hello>,
 <owslib.wps.Process dummyprocess>,
 <owslib.wps.Process wordcounter>,
 <owslib.wps.Process chomsky>,
 <owslib.wps.Process inout>,
 <owslib.wps.Process binaryoperatorfornumbers>,
 <owslib.wps.Process show_error>,
 <owslib.wps.Process multiple_outputs>,
 <owslib.wps.Process esgf_demo>,
 <owslib.wps.Process output_formats>,
 <owslib.wps.Process poly_centroid>,
 <owslib.wps.Process ncmeta>]

## Documentation about a process

With `owslib`, the process title and abstract can be obtained simply by looking at these attributes. 
For the process inputs, we need to iterate on the inputs and access their individual attributes. To facilitate this, `owslib.wps` provides the `printInputOuput` function. 

With `birdy`, just type `help(cli.hello)` and the docstring will show up in your console. With the IPython console or a Jupyter Notebook, `cli.hello?` would do as well. The docstring follows the NumPy convention. 

In [82]:
from owslib.wps import printInputOutput
p = wps.describeprocess('hello')
print("Title: ", p.title)
print("Abstract: ", p.abstract)

for inpt in p.dataInputs:
    printInputOutput(inpt)

Title:  Say Hello
Abstract:  Just says a friendly Hello.Returns a literal string output with Hello plus the inputed name.
 identifier=name, title=Your name, abstract=Please enter your name., data type=//www.w3.org/TR/xmlschema-2/#string
 Any value allowed
 Default Value: None 
 minOccurs=1, maxOccurs=1


In [83]:
 help(cli.hello)

Help on method hello in module birdy.client.base:

hello(name=None) method of birdy.client.base.WPSClient instance
    Just says a friendly Hello.Returns a literal string output with Hello plus the inputed name.
    
    Parameters
    ----------
    name : //www.w3.org/TR/xmlschema-2/#string
        Please enter your name.
    
    Returns
    -------
    output : //www.w3.org/TR/xmlschema-2/#string
        A friendly Hello from us.



## Launching a process and retrieving literal outputs

With `owslib`, processes are launched using the `execute` method. Inputs are an an argument to `execute` and defined by a list of key-value tuples. These keys are the input names, and the values are string representations. The `execute` method returns a `WPSExecution` object, which defines a number of methods and attributes, including `isComplete` and `isSucceeded`. The process outputs are stored in the `processOutputs` list, whose content is stored in the `data` attribute. Note that this `data` is a list of strings, so we may have to convert it to a `float` to use it. 

With `birdy`, inputs are just typical keyword arguments, and outputs are already converted into python objects. If there are multiple outputs, then a `namedtuple` is returned. 

In [84]:
resp = wps.execute('binaryoperatorfornumbers', inputs=[('inputa', '1.0'), ('inputb', '2.0'), ('operator', 'add')])
if resp.isSucceded:
    output, = resp.processOutputs
    print(output.data)

['3.0']


In [85]:
cli.binaryoperatorfornumbers(1, 2, operator='add')
cli.inout()

/home/david/.conda/envs/birdy/lib/python3.6/site-packages/owslib/wps.py:1326: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not bboxDataElement:


['This is just a string',
 7,
 3.14,
 True,
 datetime.time(12, 0),
 datetime.date(2012, 5, 1),
 datetime.datetime(2016, 9, 2, 12, 0, tzinfo=tzutc()),
 'scissor',
 'gentle albatros',
 'http://localhost:5000/outputs/d3ae9808-f291-11e8-8494-b052162515fb/input',
 'http://localhost:5000/outputs/d3ae9808-f291-11e8-8494-b052162515fb/input.txt',

## Retrieving outputs by references

For `ComplexData` objects, WPS servers often return a reference to the output (an http link) instead of the actual data. This is useful if that output is to serve as an input to another process, so as to avoid passing back and forth large files for nothing. 

With `owslib`, that means that the `data` attribute of the output is empty, and we instead access the `reference` attribute. The referenced file can be written to the local disk using the `writeToDisk` method. 

With `birdy`, the outputs are by default the references themselves, but it's also possible to download these references in the background and convert them into python objects. To trigger this automatic conversion, set `convert_objects` to `True` when instantating the client `WPSClient(url, convert_objects=True)`. Ini the example below, the first output is a plain text file, and the second output is a json file. The text file is converted into a string, and the json file into a dictionary. 

In [86]:
resp = wps.execute('multiple_outputs', inputs=[('count', '1')])
output, ref = resp.processOutputs
print(output.reference)
print(ref.reference)
output.writeToDisk('/tmp/output.txt')

http://localhost:5000/outputs/d4821476-f291-11e8-8494-b052162515fb/output_0.txt
http://localhost:5000/outputs/d4821476-f291-11e8-8494-b052162515fb/input.json


In [87]:
output, ref = cli.multiple_outputs(1)
print(output)
print(ref)
clic = WPSClient(url, convert_objects=True)
output, ref = clic.multiple_outputs(1)
print(output)
print(ref)

http://localhost:5000/outputs/d9e9114e-f291-11e8-8494-b052162515fb/output_0.txt
http://localhost:5000/outputs/d9e9114e-f291-11e8-8494-b052162515fb/input.json
my output file number 0
{'count': 1, 'outputs': [{'name': 'output_0.txt', 'url': 'http://localhost:5000/outputs/daf5ecec-f291-11e8-8494-b052162515fb/output_0.txt'}]}
